# Objective : Scraping the data from e-commerce webiste. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Web Scrapping/Whisky_from_Ecommerce_website


/content/drive/MyDrive/Colab Notebooks/Web Scrapping/Whisky_from_Ecommerce_website


In [4]:
!pwd

/content/drive/My Drive/Colab Notebooks/Web Scrapping/Whisky_from_Ecommerce_website


### Importing the libraries




In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
baseurl = 'https://www.thewhiskyexchange.com/'

In [7]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 RuxitSynthetic/1.0 v9410181630 t38550 ath9b965f92 altpub cvcv=2'
}

### Scrapping the products links from every pages

In [8]:
productlinks = []
for x in range(1,6):
  r = requests.get(f'https://www.thewhiskyexchange.com/c/35/japanese-whisky?pg={x}&psize=24&sort=pasc')
  soup = BeautifulSoup(r.content, 'html.parser')

  # 
  productlist=  soup.find_all('li', {'class' :'product-grid__item'})

 # Looping through each product ()
  for item in productlist:
    # extracting links through every pages
    for link in item.find_all('a', href = True):
      productlinks.append(baseurl + link['href'])

### Scraping the data from product page

In [9]:
#testlink = 'https://www.thewhiskyexchange.com/p/37325/suntory-torys-classic'

whiskylist = []
for link in productlinks:
  r = requests.get(link, headers = headers)

  soup = BeautifulSoup(r.content, 'lxml')

  #Scraping the rating, price & description  from pages
  name = soup.find('h1', class_ = 'product-main__name').text.strip()
  price = soup.find('p', class_ = 'product-action__price').text.strip()
  description = soup.find('div', class_ = 'product-main__description').text.strip()

  # Scraping the rating from pages
  try:
    rating = soup.find('span', class_ = 'review-overview__rating star-rating star-rating--35').text.strip()
  except:
    rating = 'no rating'
  
  # Scraping the reviews of products from pages
  try:
    reviews = soup.find('span', class_ = 'review-overview__count').text.strip()
  except:
    reviews = ''

  # Saving all data into dictionary
  whisky = {
      'name' : name,
      'rating' : rating,
      'reviews' : reviews,
      'description' : description,
      'price' : price
      }

# Saving the scrape data into list
  whiskylist.append(whisky)

### Saving Scraped data into dataframe

In [10]:
df = pd.DataFrame(whiskylist)
df.head(15)

,name,rating,reviews,description,price
0,Suntory Torys Classic,3.5,(2 Reviews),Suntory Torys Classic is a light and easy-drin...,£30.45
1,Suntory Toki,no rating,(30 Reviews),Toki is a blended whisky from Suntory's three ...,£31.95
2,Akashi Blended Whisky,3.5,(19 Reviews),A lesser-known whisky outside the local Japane...,£32.75
3,Hatozaki Blended Japanese Whisky,no rating,(3 Reviews),Named after the oldest stone lighthouse in Jap...,£33.45
4,Suntory Toki\nGlass Pack,no rating,,"A lovely gift set from Suntory, including a bo...",£34.95
5,Nikka Days,no rating,(1 Review),"A vibrant addition to the Nikka range, Nikka D...",£38.95
6,Mars Kasei Blended Whisky,no rating,,"Mars Kasei blended whisky is a soft, rounded w...",£42.95
7,Nikka From the Barrel,no rating,(107 Reviews),Nikka from the Barrel is big-boned and burstin...,£42.95
8,Togouchi Premium Blended Whisky,no rating,(10 Reviews),"Although blended in Japan, Togouchui Premium W...",£44.95
9,Nikka Pure Malt Red,no rating,(3 Reviews),Blended with Japanese malt from Nikka's Yoichi...,£46.45


In [11]:
df.to_csv('whisky_data.csv', index = False) # saving the data into disk